In [1]:
import os
import pandas as pd

In [2]:
# Figure out path to pwd and data dir
_pwd = os.path.abspath("..")
_data_dir = os.path.join(_pwd, "data")

In [4]:
# Point your csv file here.
# Data can be collected from https://github.com/serval-snt-uni-lu/Piggybacking/blob/master/ground-truth/all-pairs.csv
_apk_csv = os.path.join(_data_dir, 'piggyback-all-pairs.csv')

In [5]:
API_KEY=open(os.path.join(_pwd, 'ANDROZOO_API_KEY.txt'),"r").read()
print(API_KEY)

e03b3548509b2b94d502314ca0f5cf7012c0357b18e6bea07071dec729c00c25


In [5]:
# Read the CSV
apk_df = pd.read_csv(_apk_csv)

In [8]:
apk_df

,ORIGINAL_APP,PIGGYBACKED_APP
0,00575DE650F78413C91E8E613ADF9099812F325AA9CF36...,A55EA5807203CE31470D1905ED30F9267AE3459809DDA2...
1,006009DB95496D4E42281EB18C6E5CA81132DE1489F690...,B5610062146D79CA6CDFDCA2F42555AEE3C5317FDDA76B...
2,009D7E035C1A41CCEA8F4332506F3F65B14C673750C1FD...,03E4B7D80A9CB3C2169F2CF4BAD1EDFBE14D5A3EB5DFC7...
3,00B04CDA4F1C6F3A838B0303EC6142D7420164555F08BF...,A7CB7B66EB8AF4F27DC856C942AE2B35505B848A4C891E...
4,00C381F40F4084F98CE6C7A9A32A1BBC890123E9244F77...,2DC27146F98E2C993F38E0AF9A1604430C98E8861353A2...
...,...,...
1492,FF2C4EEFD1DF8612214CA7C301067ECE8259161A11BA4A...,88DD0EDE396923D0C3C059C8325673E9D24A65FAA1DCF8...
1493,FF4C46846ACBFFC38E8FFBBDA515EC226196260055D062...,1F7BEAA3824CF73D6450A48EEAC46307FC58E52A65AB1C...
1494,FF796142209D76DC073DD65F06F2362685E02B689418D2...,95ED7B2A52B9491D5BF3518E3C6A6D1EC613B261758F87...
1495,FFDE8B674AE70E27D04303EAEB8B6DA2B28BA5341848E2...,DB2CB6BC378FA5ED47047D5AA332F69A4F4663CDA3C9AF...


In [6]:
# Find number of malicious apks for each benign apk.
apk_df['ORIGINAL_APP'].value_counts()

CE81D0A16CDCA3C6CCEB9A6607A47DC9DEBB7B7C9B84638263580F43427D4DFF    56
B45EB1CB7A931D963970F96150675F73926A3140D5376FF7A355FB07F4F55992    56
1DAF15DFCE13770C5CA76FAE2E1EE93EF62E56A267A6BE87D27C088FA9790870     3
CCC96109AB08504F4D1555A2B0F79AEB61D500463696E28159845FF65686AE03     3
A932CB38A67021F65A4FBA882E5B25FFA584036CD8BC617307E59F73F1398C63     3
                                                                    ..
51DC034AB4E1BC60F615F4B64D2789338D42956F37C238196071C6DBDAAE6AF4     1
86443B8F2A6FBE9867F264A933F011CABA9F409032B7E305A0BF25934B79FA36     1
D73500CA6776ABBDD265764B40C1B360888A4DF126A7647CFE171522AA15EB3B     1
C1EBC944969194FD783C7B732418F7B81D763C246A80D1355BC336CA5DE32D3B     1
9FB15E645F7CC65AFC9BDE72C92F63DDA44E0E6F4C4AA19D7DD4402FE3A3716B     1
Name: ORIGINAL_APP, Length: 1355, dtype: int64

In [9]:
# Request the AndroZoo API for the apk and write it to a directory
# 
# params: 
#     api_key: Private key to the AndroZoo API
#     sha256: sha256 of the apk 
#    data_dir: directory to save the file. 
def request_apk_andro(api_key, sha256, data_dir):
    import requests
    url = "https://androzoo.uni.lu/api/download?"

    packaged_url = url + 'apikey=' + api_key + '&sha256=' + sha256
    print("Requesting", packaged_url)
    r = requests.get(packaged_url)
    
    filename = os.path.join(data_dir, sha256 + ".apk")
    with open(filename, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)

In [10]:
# Gets all malicious apks for a corresponding benign apk. 
# The benign apks are stored in 
# 
# params: 
#     df: DataFrame with columns "ORIGINAL_APP" and "PIGGYBACKED_APP"
#     b_sha256: benign apk's sha256
#     data_dir: directory to save the file. 
def get_all_malicious(df, b_sha256, data_dir):
    malicious_sha_list = df.loc[df['ORIGINAL_APP'] == b_sha256]['PIGGYBACKED_APP']
    
    benign_folder_path = os.path.join(data_dir, "benign")
    print(benign_folder_path)

    if not os.path.isdir(benign_folder_path):
        os.makedirs(benign_folder_path)

    request_apk_andro(API_KEY, str(b_sha256), benign_folder_path)
    
    malicious_folder_path = os.path.join(data_dir, "malicious")
    print(malicious_folder_path)
    if not os.path.isdir(malicious_folder_path):
        os.makedirs(malicious_folder_path)
    
    for malicious_sha256 in malicious_sha_list:
        request_apk_andro(API_KEY, str(malicious_sha256), malicious_folder_path)

In [12]:
# for idx, sha256 in enumerate(apk_df['ORIGINAL_APP'].unique().tolist()[:1]):
sha256 = "86443B8F2A6FBE9867F264A933F011CABA9F409032B7E305A0BF25934B79FA36"
get_all_malicious(apk_df, sha256, os.path.join(_data_dir, f"app_{11}"))

/Users/jarus/Work/r2callgraph4apk/data/app_11/benign
Requesting https://androzoo.uni.lu/api/download?apikey=e03b3548509b2b94d502314ca0f5cf7012c0357b18e6bea07071dec729c00c25&sha256=86443B8F2A6FBE9867F264A933F011CABA9F409032B7E305A0BF25934B79FA36
/Users/jarus/Work/r2callgraph4apk/data/app_11/malicious
Requesting https://androzoo.uni.lu/api/download?apikey=e03b3548509b2b94d502314ca0f5cf7012c0357b18e6bea07071dec729c00c25&sha256=1BEC3EC4B07E41F99B8D62CB4C7323B0702ADC6F5D0F576E1DD32B035D1636FE
